# Task 4 

In [1]:
# Imports
import pandas as pd
import numpy as np

#### a) 

Here, we will use the least square (LS) principle, since the variables comes from two different distributions, namely product 1 and product 2.

We let P denote the production rate, $x_1$ the product type (let 0 be product 1 and 2 be product 1, this works due to the assumption that $\beta_1$ is a constant that represents the expected difference in production rates for the two products) and $x_2$ be the number of extra workers (this works because we assume an extra operator will have the same effect in terms of expected increase in production rate for the two producst).

In [2]:
# Data
dic = {'P': [11.5, 10.8, 9, 9.7, 7.8, 7.5, 7.8, 11.6, 11.7, 9.6, 7.9, 7.4, 7.6],
      'x1': [0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1],
      'x2': [1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0]}
df = pd.DataFrame(dic)
display(df)

,P,x1,x2
0,11.5,0,1
1,10.8,0,1
2,9.0,1,1
3,9.7,0,0
4,7.8,1,0
5,7.5,1,0
6,7.8,1,0
7,11.6,0,1
8,11.7,0,1
9,9.6,1,1


#### b) 

**Step 1 **

To implement the LS algorithm, we reuse the code from assignment 7 problem 7.4

In [3]:
#Setup
X = np.zeros((len(df['x1']), 3))
X[:,0] = 1
X[:,1] = df['x1']
X[:,2] = df['x2']
y = np.array(df['P']).transpose()
XT = X.transpose()
#calc beta_hat
def calc_beta_hat(X = X, XT = XT, y = y):
    p1 = np.linalg.inv(np.dot(XT,X))
    p2 = np.dot(p1, XT)
    beta_hat = np.dot(p2, y)
    return beta_hat
    
beta_hat = calc_beta_hat(X, XT, y)
# print(beta_hat)
#print results
for i in range(len(beta_hat)):
    print(f"beta_{i} = {beta_hat[i]:.4f}")

beta_0 = 9.9729
beta_1 = -2.4075
beta_2 = 1.3589


Now that the function 
$$\hat{y_i} = \hat\beta_0 + \hat\beta_1x_{i,1} + \hat\beta_2x_{i,2}$$
is calculated, we can proceed with the seven steps described on page 78 in the compendium to calculate the residuals and use them as input in the method of moments to estimate the PERT parameters.

In [4]:
y_hat = np.dot(X, beta_hat)
print(y_hat)

[11.3317757  11.3317757   8.92429907  9.9728972   7.56542056  7.56542056
  7.56542056 11.3317757  11.3317757   8.92429907  8.92429907  7.56542056
  7.56542056]


**Step 3:**

In [5]:
e_hat = np.array(df['P']) - y_hat
print(e_hat)

[ 0.1682243  -0.5317757   0.07570093 -0.2728972   0.23457944 -0.06542056
  0.23457944  0.2682243   0.3682243   0.67570093 -1.02429907 -0.16542056
  0.03457944]


**Step 4:** Here we apply the same approach as in **Problem 7.2** (Method of moments), but with the residuals $\epsilon$

In [6]:
xs = e_hat
x_avg = np.mean(xs)
n = len(xs)
var = 1/(n-1)*np.sum(np.array([(x - x_avg)**2 for x in xs]))
s = np.sqrt(var)
print(f"Average = {x_avg:.2f}")
print(f"S (stdev) = {s:.2f}")

Average = 0.00
S (stdev) = 0.43


Now we need to calculate L-, M- and H-values.

In [7]:
def calc_H(xs = xs, x_avg = x_avg, S = s):
    xmax = np.max(xs)
    xmin = np.min(xs)
    return x_avg + S*np.sqrt(7*(xmax - x_avg)/(x_avg - xmin))

def calc_L(H, xs = xs, x_avg = x_avg):
    xmax = np.max(xs)
    xmin = np.min(xs)
    return x_avg - (H - x_avg)*(x_avg - xmin)/(xmax - x_avg)

def calc_M(L, H, xs = xs, x_avg = x_avg):
    return (6*x_avg - L - H)/4

In [8]:
H_hat = calc_H(xs = xs, x_avg = x_avg, S = s)
L_hat = calc_L(H_hat, xs = xs, x_avg = x_avg)
M_hat = calc_M (L_hat, H_hat, xs = xs, x_avg = x_avg)
print(f"L_hat = {L_hat:.2f}")
print(f"M_hat = {M_hat:.2f}")
print(f"H_hat = {H_hat:.2f}")

L_hat = -1.41
M_hat = 0.12
H_hat = 0.93


**Step 7:** Calculate L, M, H by adding y0 to L_hat, M_hat and H_hat respectively 

In [9]:
y0 = df['P'][4]
L, M, H = L_hat + y0, M_hat + y0, H_hat + y0
print(f"L = {L:.2f}")
print(f"M = {M:.2f}")
print(f"H = {H:.2f}")

L = 6.39
M = 7.92
H = 8.73


#### c)


Mean value og variance av hva? Man får jo ikke noe profit av å produsere på maskin B direkte, skal man legge til den nye distribusjonen og regne ut tilsvarende tidligere oppgaver?